# Setup

In [ ]:
!pip install jupyter-dash -q

In [ ]:
import plotly.express as px
from jupyter_dash import JupyterDash
import dash_core_components as dcc
import dash_html_components as html
from dash.dependencies import Input, Output

In [ ]:
import pandas as pd
import numpy as np

# Loading Some Data

I used the data from https://drive.google.com/drive/u/1/folders/1dxwRUNX0a3iQhbvzJrOpvutKfadoBooS (BD11 spark ml notebook)

In [ ]:
df = pd.read_csv('energy_data_ml.csv').drop('Unnamed: 0', axis=1)
print(df.shape)
df.head()

We might want to plot the relationship between Surface Area and Cooling Load. In pandas, we can do:

In [ ]:
df.plot(kind='scatter', x='Surface_Area', y='Cooling_Load')

But what if we wanted to share this data and make it interactive? Enter Plotly/Dash

# Minimal Example

In [ ]:
# Build App
app = JupyterDash(__name__)
app.layout = html.Div([
    html.H1("JupyterDash Demo 1 - Use HTML components!"),
    dcc.Graph(id='graph1',
      figure=px.scatter(
          df, # Specify the dataframe
          x="Surface_Area", 
          y="Cooling_Load",
          title="Surface Area vs Cooling Load",
          color='Surface_Area' # We can add color or size options...
      )
    ),
])

# Run app and display result inline in the notebook
app.run_server(mode='inline')

That's pretty cool already, and since it's a flask app under the hood it's very easy to host somewhere (eg pythonanywhere). But where plotly shines is in interactivity:

# Now make it interactive!

In [ ]:
# Build App
app = JupyterDash(__name__)
app.layout = html.Div([
    html.H1("JupyterDash Demo 2 - Callbacks and interactivity"),
    dcc.Graph(id='graph1'), # We'll update the figure in the callback
    html.Label([
        "X Axis",
        dcc.Dropdown(
            id='x-axis-dropdown', clearable=False,
            value='Surface_Area', options=[
                {'label': c, 'value': c}
                for c in df.columns
            ])
    ]),
])

# Callbacks!
# Define callback to update graph
@app.callback(
    Output('graph1', 'figure'), # What are we updating?
    [Input("x-axis-dropdown", "value")] # What triggers an update?
)
def update_figure(chosen_x):
    figure=px.scatter(
          df, # Specify the dataframe
          x=chosen_x, 
          y="Cooling_Load",
          title=chosen_x + " vs Cooling Load",
          color='Surface_Area' # We can add color or size options...
      )
    return figure

# Run app and display result inline in the notebook
app.run_server(mode='inline')

# Multiple figures

There is no reason we can't make multiple figures and arrange them however we like. Here's an example:

In [ ]:
# Build App
app = JupyterDash(__name__, external_stylesheets=['https://codepen.io/chriddyp/pen/bWLwgP.css'])
app.layout = html.Div([
    html.H1("JupyterDash Demo 3 - Multiple Figures!"),
    html.Div(className="row", style={ 'width':'100%'}, children=[
        html.Div(
            dcc.Graph(id='graph2',
              figure=px.plot(
                  df, # Specify the dataframe
                  x="Roof_Area", 
                  y="Cooling_Load",
                  title="Roof Area vs Cooling Load"
              ),
            ), style={'display': 'inline-block', 'width':'49%'} # We can use HTML/CSS styling)
        ),
        html.Div(
            dcc.Graph(id='graph3',
              figure=px.scatter(
                  df, # Specify the dataframe
                  x="Wall_Area", 
                  y="Cooling_Load",
                  title="Wall Area vs Cooling Load"
              ),
            ), style={'display': 'inline-block', 'width':'49%'} # We can use HTML/CSS styling)
        ),
    ]),
    
    dcc.Graph(id='graph1',
      figure=px.scatter(
          df, # Specify the dataframe
          x="Surface_Area", 
          y="Cooling_Load",
          title="Surface Area vs Cooling Load",
          color='Surface_Area' # We can add color or size options...
      )
    ),
])

# Run app and display result inline in the notebook
app.run_server(mode='inline')

In [ ]:
# To run this and view in a separate tab:
app.run_server(mode='external')